In [1]:
from tqdm import tqdm # Barra de progreso


import transformers
transformers.__version__

'4.20.1'

In [3]:
#!pip install protobuf==3.20.*

In [5]:
model_name = "finiteautomata/beto-sentiment-analysis"

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/469k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


### _Test con los títulos de noticias_

In [9]:
import pandas as pd
df = pd.read_csv("data/dataset-loslagos-8months.csv", 
                 sep=";", 
                 engine='python')
df.columns = ["date", "media_outlet", "url", "title", "text"]
print(len(df))
df.head()

29598


,date,media_outlet,url,title,text
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At..."


In [10]:
sub = df[:100].copy()
sub['title_label'] = ""
sub['title_score'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - sentiment', total=sub.shape[0]):
    
    # Analizamos su sentimiento en inglés
    sentiment_value = nlp(row['title'])
    
    # Insertamos en dataframe
    sub.at[index, "title_label"] = sentiment_value[0].get('label')
    sub.at[index, "title_score"] = sentiment_value[0].get('score')


sub rows - sentiment: 100%|██████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.89it/s]


In [11]:
pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)
sub[['title', "title_label", 'title_score']]

,title,title_label,title_score
0,Reconocen a guardaparques de la Región de Los Lagos como actores claves en la conservación,POS,0.953994
1,Con nuevos materiales comienza plan piloto en Saltos del Petrohué,NEU,0.927709
2,Centro de Salud Familiar CESFAM Puerto Varas invita a prevenir el Cáncer Cervicouterino con extensión horaria para exámenes PAP,NEU,0.994852
3,Alcalde Tomás Gárate presidió por primera vez la octava sesión del Consejo Comunal de la Sociedad Civil COSOC Puerto Varas,NEU,0.944004
4,Galería de Arte Machacoya realizará remate de obras de artistas de la zona,NEU,0.995532
5,Municipio llamará a licitación construcción de pasarela peatonal en Los Notros,NEU,0.99521
6,3era Mesa de Reactivación Económica circunscribe su radio de acción,NEU,0.972455
7,Gremio médico rechaza cierre de camas críticas implementadas en Hospital Ancud,NEU,0.973984
8,Asistentes de educación municipal en Osorno verán incrementadas sus remuneraciones,NEU,0.992781
9,CONADI llama a renovar directivas de comunidades indígenas,NEU,0.990752


### Estructura de una noticia
Según [Wikipedia](https://es.wikipedia.org/wiki/Noticia), una noticia está compuesta por las siguientes secciones:

- **Epígrafe o antetítulo**: Se sitúa al lector en el contexto sobre el que se va a hablar.
- **El título**: Se trata de resumir la información en un número limitado de palabras. El título debe despertar el interés del público, tomando en cuenta, dos elementos clave de la noticia como lo son la precisión y la concisión.
- **Bajada o subtítulo**: Aporta información adicional que resume brevemente lo sucedido.
- **Entradilla, copete o lead**: Es el primer párrafo de la noticia, en el cual se deben concentrar los datos más relevantes del hecho y/o del acontecimiento. Si el título atrae al público, el lead debe confirmar su interés.
- **El cuerpo**: Se desarrolla a profundidad lo descrito en el lead. En esta parte de la noticia, se deben ir desglosando los datos del hecho en un orden de mayor a menor importancia, siguiendo la estructura de la pirámide invertida
- **Cierre o remate**: Es el último párrafo de la nota, denominado remate, tiene por función “cerrar” la noticia; darle a entender al receptor que la noticia allí precisamente se concluye.


En base a lo anterior, se diseñará una estrategia que permita acercarnos más en la obtención del grado de polaridad de una noticia.

- <u>Estrategia 1</u>: Analizar el lead (_Puesto que no podemos exceder cierta cantidad de caracteres en el análisis de sentimiento, es necesario particionar el content_)

In [15]:
sub['lead_label'] = ""
sub['lead_score'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - lead_sentiment', total=sub.shape[0]):
    
    # Obtenemos la primera oración.
    lead = row['text'].split(".")[0] 

    # Analizamos su sentimiento en inglés
    sentiment_value = nlp(lead)
    
    # Insertamos en dataframe
    sub.at[index, "lead_label"] = sentiment_value[0].get('label')
    sub.at[index, "lead_score"] = sentiment_value[0].get('score')

sub rows - lead_sentiment: 100%|█████████████████████████████████████████████████████| 100/100 [00:08<00:00, 12.17it/s]


In [16]:
sub[['title', "title_label", 'title_score', "lead_label", 'lead_score']]

,title,title_label,title_score,lead_label,lead_score
0,Reconocen a guardaparques de la Región de Los Lagos como actores claves en la conservación,POS,0.953994,POS,0.986749
1,Con nuevos materiales comienza plan piloto en Saltos del Petrohué,NEU,0.927709,POS,0.997575
2,Centro de Salud Familiar CESFAM Puerto Varas invita a prevenir el Cáncer Cervicouterino con extensión horaria para exámenes PAP,NEU,0.994852,NEU,0.991337
3,Alcalde Tomás Gárate presidió por primera vez la octava sesión del Consejo Comunal de la Sociedad Civil COSOC Puerto Varas,NEU,0.944004,NEU,0.898813
4,Galería de Arte Machacoya realizará remate de obras de artistas de la zona,NEU,0.995532,POS,0.780594
5,Municipio llamará a licitación construcción de pasarela peatonal en Los Notros,NEU,0.99521,NEU,0.963101
6,3era Mesa de Reactivación Económica circunscribe su radio de acción,NEU,0.972455,NEU,0.826101
7,Gremio médico rechaza cierre de camas críticas implementadas en Hospital Ancud,NEU,0.973984,NEG,0.71099
8,Asistentes de educación municipal en Osorno verán incrementadas sus remuneraciones,NEU,0.992781,POS,0.995816
9,CONADI llama a renovar directivas de comunidades indígenas,NEU,0.990752,NEU,0.981928


In [20]:
aux = sub.loc[sub.title_label == 'NEG']
aux[['title', 'title_score']]

,title,title_score
10,"Tasa de desocupación en la región de Los Lagos fue de un 3,4%",0.995607
16,Estudiantes denuncian que CFT Estatal Los Lagos no les da un lugar estable donde estudiar,0.998562
28,Joven ciclista es atropellada en Osorno,0.992902
34,Puqueldón avanza a apertura en el fin del Estado de Excepción,0.892468
40,Acusan que obrero intentó cortarle el cuello a su pareja: quedó en prisión,0.845691
48,"Rentas vitalicias: empresas y CPC, en picada contra otro giro",0.996605
51,Contraloría rechaza pedido para enviar personal de las Fuerzas Armada a la Macrozona Sur,0.961464
53,El gol no quiso nada con Colo Colo y su derrota deja el torneo al rojo,0.997701
69,Anuncian estreno de álbum perdido de David Bowie,0.537465
73,Pucatrihue a oscuras: vecinos se quejan por falta de iluminación en vía principal,0.990974


In [22]:
aux = sub.loc[sub.title_label == 'POS']
aux[['title', 'title_score']]

,title,title_score
0,Reconocen a guardaparques de la Región de Los Lagos como actores claves en la conservación,0.953994
21,Comunidad valora compromiso y acciones implementadas en el Hospital Comunitario de Queilen,0.973287
24,Cesfam Entre Lagos mejora su infraestructura.,0.920584
25,VIALIDAD Y GOBERNADOR REGIONAL COMPROMETEN IMPORTANTES AVANCES EN CONECTIVIDAD PARA PURRANQUE,0.993801
29,Positiva evaluación de Carabineros tras primera noche sin toque de queda en la provincia de Llanquihue,0.923524
32,Más de 2 mil 800 postulaciones para fondos concursables recibió San Juan de La Costa,0.736428
68,"""Bridgerton"" es la serie más vista en la historia de la plataforma Netflix",0.99602
78,Escuela Ana Aichele cumple 52 años de vida,0.993535
83,Con seminario y visita a parque chilote parte la Semana del Turismo,0.965341


In [23]:
aux = sub.loc[sub.title_label == 'NEU']
aux[['title', 'title_score']]

,title,title_score
1,Con nuevos materiales comienza plan piloto en Saltos del Petrohué,0.927709
2,Centro de Salud Familiar CESFAM Puerto Varas invita a prevenir el Cáncer Cervicouterino con extensión horaria para exámenes PAP,0.994852
3,Alcalde Tomás Gárate presidió por primera vez la octava sesión del Consejo Comunal de la Sociedad Civil COSOC Puerto Varas,0.944004
4,Galería de Arte Machacoya realizará remate de obras de artistas de la zona,0.995532
5,Municipio llamará a licitación construcción de pasarela peatonal en Los Notros,0.99521
6,3era Mesa de Reactivación Económica circunscribe su radio de acción,0.972455
7,Gremio médico rechaza cierre de camas críticas implementadas en Hospital Ancud,0.973984
8,Asistentes de educación municipal en Osorno verán incrementadas sus remuneraciones,0.992781
9,CONADI llama a renovar directivas de comunidades indígenas,0.990752
11,Se constituyó nuevo Comité para la Gestión del Riesgo de Desastres (COGRID),0.981968
